In [ ]:
import pandas as pd
import numpy as np

from PIL import Image, ImageOps

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from skimage.filters import sobel
from skimage.io import imread
from skimage import morphology
from skimage.feature import canny
from scipy import ndimage as ndi
from skimage.segmentation import find_boundaries, mark_boundaries, slic

In [ ]:
locations = pd.read_hdf("data/backup/complete_paths.h5", "data")

In [ ]:
locations = locations[((locations['ice'] > 0) & (locations['ice'] <= 100)) | (locations["ice"] == 255)]

In [ ]:
def get_conc(image):
    try:
        im = imread(image, as_grey=True, plugin="pil")[375:800, :]*255
        elevation_map = sobel(im)
        #print(image, np.mean(im))
        
        # bright enough?
        if np.mean(im) < 110:
            hist = np.histogram(im, bins=np.arange(0, 256), normed=True)
            
            # number of pixel values over 10% of max(hist)
            perc_over = (hist[0] > (0.1*np.amax(hist[0]))).sum()
            # more ice than water
            if perc_over > 120:
                filtered_hist = hist[1][hist[0] > 0.01]
            else:
                # find ideal cut in hist, if more water than ice: ice is over 150 (gray scale)
                border = hist[1][(hist[0] < 0.1*np.mean(hist[0])) & (hist[1][1:] > 150)][0]
                filtered_hist = hist[1][border:][(hist[0][border:] > 0.0001)]
            
            # find filtering borders in the brightness
            low_ice = np.min(filtered_hist)
            high_ice = np.max(filtered_hist)
            low_water = np.floor(low_ice/10)*10
            high_water = np.ceil(high_ice/10)*10

            markers = np.zeros_like(im)
            markers[(im < low_water) | (im > high_water)] = 1 # ship, water
            markers[(im > low_ice) & (im < high_ice)] = 3

            segmentation = morphology.watershed(elevation_map, markers)
            return (segmentation == 3).sum()/(im.shape[0]*im.shape[1])
        else:
            return 0.0
    except:
        return np.nan

In [ ]:
test_set = locations[locations["ice"] != 255][:10]
test_set["predicted"] = test_set["image"].apply(get_conc)
test_set

In [ ]:
#test_image = "data/images/remote/20111120-2101.jpeg"
#test_image = "data/images/remote/20150908-2001.jpeg"
test_image = "/media/lucas/data/images/20140819-1001.jpeg"

In [ ]:
Image.open(test_image).convert("L")

In [ ]:
im = imread(test_image, as_grey=True, plugin="pil")[375:800, :]*255
elevation_map = sobel(im)
elevation_map[elevation_map < 10] = 0

markers = np.zeros_like(im)
#markers[(im < 180) | (im > 250)] = 1 # ship, water, 86
#markers[(im > 180) & (im < 250)] = 3
markers[(im < 100)] = 1 # ship, water, 98
markers[(im > 100)] = 3

segmentation = morphology.watershed(elevation_map, markers)
#segmentation = slic(elevation_map, enforce_connectivity=False, sigma=5, n_segments=10)
(segmentation == 3).sum()/(im.shape[0]*im.shape[1])

In [ ]:
background = Image.open(test_image).crop((0,375,1280,800)).convert("RGBA")
im2 = Image.fromarray(elevation_map/np.max(elevation_map)*255).convert("L")

layer = Image.new('RGBA', background.size, color=(120,255,0,255))

Image.composite(background, layer, ImageOps.invert(im2))

In [ ]:
Image.fromarray(segmentation.astype(np.uint8)*40).convert("L")

In [ ]:
hist = np.histogram(im, bins=np.arange(0, 256), normed=True)
plt.plot(hist[1][:-1], hist[0], lw=2) 

In [ ]:
fill_ice = ndi.binary_fill_holes(segmentation-1)
#bounds = find_boundaries(fill_ice)
bounds = mark_boundaries(im, fill_ice)
Image.fromarray(bounds.astype(np.uint8)*255)

In [ ]:
filtered_hist = hist[1][(hist[0] > 0.0001)]
low_ice = np.min(filtered_hist)
high_ice = np.max(filtered_hist)
low_water = np.floor(low_ice/10)*10
high_water = np.ceil(high_ice/10)*10
print(low_ice, high_ice, low_water, high_water)

In [ ]:
hist[1][(hist[0] < 0.1*np.mean(hist[0])) & (hist[1][1:] > 150)][0]

In [ ]:
(hist[0] > (0.1*np.amax(hist[0]))).sum()

In [ ]:
 np.mean(hist[0])

In [ ]:
locations[locations.index.strftime("%Y%m%d") == "20111120"]